## Test Pipeline for Binance Crypto Data

This notebook contains test runs for extracting, transforming and loading cryptocurrency data from Binance API to a PostgreSQL database. The PostgreSQL table and schema to be specified will the used for testing purposes only and will be hosted on Aiven.


In [46]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv()
DB_URI = os.getenv("DB_URI")
BASE_URL = 'https://api.binance.com'
symbols = [
    "BTCUSDT",   
    "ETHUSDT",
    "BNBUSDT",
    "XRPUSDT",
    "SOLUSDT",
    "ADAUSDT",
    "DOGEUSDT",
    "MATICUSDT",
    "DOTUSDT",
    "AVAXUSDT"
]

In [ ]:
# Get depth in the order book
def get_depth(BASE_URL, symbols, DB_URI): # add DB_URI when loading
    coin_data = []
    url = f"{BASE_URL}/api/v3/depth"
    for s in symbols:
        params = {'symbol': s, 'limit': 10}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            for side, orders in [('bid', data['bids']), ('ask', data['asks'])]:
                for price, qty in orders:
                    coin_data.append({
                        'update_id': data['lastUpdateId'],
                        'symbol': s,
                        'side': side,
                        'price': float(price),
                        'quantity': float(qty),
                        'captured_at': int(time.time() * 1000)
                    })
        else:
            print(f"Error fetching order book: {response.status_code}, {response.text}")
    
    df = pd.DataFrame(coin_data)
    df["captured_at"] = pd.to_datetime(df["captured_at"], unit='ms')
    try:
        engine = create_engine(DB_URI)
        df.to_sql('order_book', con=engine, schema='binance_test', index=False, if_exists='append')
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Error loading order book into db: {e}")

get_depth(BASE_URL, symbols, DB_URI)

update_id               int64
symbol                 object
side                   object
price                 float64
quantity              float64
captured_at    datetime64[ns]
dtype: object

In [34]:
# get recent trades from API
def get_recent_trades(BASE_URL, symbols, DB_URI):
    coin_data = []
    url = f"{BASE_URL}/api/v3/trades"
    for s in symbols:
        params = {'symbol': s, 'limit': 10}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            for d in data:
                coin_data.append({
                    'symbol': s,
                    'trade_id': d["id"],
                    'price': d["price"],
                    'qty': d['qty'],
                    'time': d['time']
                })
        else:
            print(f"Error fetching recent trades: {response.status_code}, {response.text}")
    df = pd.DataFrame(coin_data)
    df["price"] = df["price"].astype(float)
    df["qty"] = df["qty"].astype(float)
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    try:
        engine = create_engine(DB_URI)
        df.to_sql('recent_trades', con=engine, schema='binance_test', index=False, if_exists='append')
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Loading error into databases: {e}")
    
get_recent_trades(BASE_URL, symbols, DB_URI)

Data loaded successfully!


In [45]:
# get klines
def get_klines(BASE_URL, symbols, DB_URI):
    coin_data = []
    url = f"{BASE_URL}/api/v3/klines"
    for s in symbols:
        params = {'symbol': s, 'limit': 10, 'interval': '30m'}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            for d in data:
                coin_data.append({
                        'symbol': s,
                        'open_time': d[0],
                        'open_price': float(d[1]),
                        'high_price': float(d[2]),
                        'low_price': float(d[3]),
                        'close_price': float(d[4]),
                        'volume': float(d[5]),
                        'close_time': d[6],
                        'num_trades': d[8]
                    })
        else:
            print(f"Error fetching recent trades: {response.status_code}, {response.text}")

    df = pd.DataFrame(coin_data)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    try:
        engine = create_engine(DB_URI)
        df.to_sql("klines", con=engine, schema='binance_test', index=False, if_exists='append')
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Loading kline data into database: {e}")

get_klines(BASE_URL, symbols, DB_URI)

Data loaded successfully!


In [11]:
# 24H ticker
def get_daily_ticker(BASE_URL, symbols):
    url = f"{BASE_URL}/api/v3/ticker/24hr"
    for s in symbols:
        params = {'symbol': s, 'type': 'MINI'}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            print(response.json())
        else:
            print(f"Error fetching recent trades: {response.status_code}, {response.text}")

get_daily_ticker(BASE_URL, symbols)

{'symbol': 'BTCUSDT', 'openPrice': '102288.89000000', 'highPrice': '106074.20000000', 'lowPrice': '99613.33000000', 'lastPrice': '105029.19000000', 'volume': '27159.99814000', 'quoteVolume': '2820413769.36626970', 'openTime': 1750687085013, 'closeTime': 1750773485013, 'firstId': 5037656134, 'lastId': 5042722947, 'count': 5066814}
{'symbol': 'ETHUSDT', 'openPrice': '2291.00000000', 'highPrice': '2449.14000000', 'lowPrice': '2188.00000000', 'lastPrice': '2420.52000000', 'volume': '810189.56280000', 'quoteVolume': '1905060216.95838500', 'openTime': 1750687086012, 'closeTime': 1750773486012, 'firstId': 2560049745, 'lastId': 2564766471, 'count': 4716727}
{'symbol': 'BNBUSDT', 'openPrice': '626.50000000', 'highPrice': '642.79000000', 'lowPrice': '617.02000000', 'lastPrice': '640.02000000', 'volume': '227951.94200000', 'quoteVolume': '144276634.63087000', 'openTime': 1750687086084, 'closeTime': 1750773486084, 'firstId': 1082596182, 'lastId': 1083405841, 'count': 809660}
{'symbol': 'XRPUSDT', 

In [53]:
# get latest prices
def get_latest_prices(BASE_URL, symbols, DB_URI):
    coin_data = []
    url = f"{BASE_URL}/api/v3/ticker/price"
    for s in symbols:
        params = {'symbol': s}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            coin_data.append({
                'symbol': s,
                'price': data["price"],
                'time': int(time.time() * 1000)
            })
        else:
            print(f"Error fetching recent trades: {response.status_code}, {response.text}")
    df = pd.DataFrame(coin_data)
    df["price"] = df["price"].astype(float)
    df["time"] = pd.to_datetime(df["time"], unit='ms')
    try:
        engine = create_engine(DB_URI)
        df.to_sql('latest_prices', con=engine, schema='binance_test', index=False, if_exists='append')
        print("Data loaded successfully!")
    except Exception as e:
        print(f"Loading latest prices to db: {e}")

get_latest_prices(BASE_URL, symbols, DB_URI)

Data loaded successfully!
